In [1]:
## The DSA needs to have appropriate metadata in order to view a multiplex image appropriately in the faceted search viewer
import girder_client
import tqdm, csv, io, gcSecrets
import pandas as pd
%load_ext autoreload
%autoreload 2
import mxifHelpers as hlprs

gc= girder_client.GirderClient(apiUrl=gcSecrets.styxApiUrl)
gc.authenticate(apiKey=gcSecrets.styxApiToken)

{'_id': '5cacfa86e62914004a60e139'}

In [22]:
vandyMxIfFolder = "61fbe2274874d4ffdf1c854c"
### Apply metadata to multiframe images
## For now I will assume that the metadata for frames are in a 
metadataFolderId = '61fd96a54874d4ffdf694ea2'
if 'vmdCache' not in locals():
    vmdCache = hlprs.generateVandyMetadataDict(metadataFolderId,gc)
     
## Get the list of items in the folder, and then I need to check their metadata and see if they have frames like I want
for i in tqdm.tqdm( gc.listItem(vandyMxIfFolder)):
    checkForLargeImage( i)
    ## Commong key between filename and metadata name... no csv or .tif
    dataFileId = i['name'].replace(".tif","")
    if dataFileId in vmdCache:
        analyzeIoParams(i, csvData=vmdCache[dataFileId])
    else:
        print("Missing metadata for",dataFileId)
    
### Need to look for frames.. if not I need to force it to recreate the item..    



0it [00:00, ?it/s]

1it [00:00,  1.13it/s]

2it [00:01,  1.31it/s]

3it [00:01,  1.55it/s]

4it [00:02,  1.67it/s]

5it [00:02,  1.71it/s]

6it [00:03,  1.83it/s]

7it [00:03,  1.95it/s]

8it [00:04,  2.08it/s]

9it [00:04,  1.97it/s]

10it [00:05,  2.04it/s]

11it [00:06,  1.28it/s]

12it [00:07,  1.47it/s]

13it [00:07,  1.46it/s]

14it [00:08,  1.46it/s]

15it [00:08,  1.64it/s]

16it [00:09,  1.71it/s]

17it [00:09,  1.68it/s]

18it [00:10,  1.66it/s]

19it [00:11,  1.63it/s]

20it [00:11,  1.64it/s]

21it [00:12,  1.78it/s]

22it [00:12,  1.80it/s]

23it [00:13,  1.94it/s]

24it [00:13,  2.10it/s]

25it [00:14,  2.17it/s]

26it [00:14,  2.20it/s]

27it [00:14,  2.20it/s]

28it [00:16,  1.25it/s]

29it [00:17,  1.25it/s]

30it [00:17,  1.46it/s]

31it [00:18,  1.66it/s]

32it [00:18,  1.81it/s]

33it [00:19,  1.92it/s]

34it [00:19,  2.03it/s]

35it [00:19,  2.13it/s]

36it [00:20,  2.22it/s]

37it [00:20,  2.22it/s]

38it [00:21,  2.29it/s]

39it [00:21,  2.19it/s]

40it [00:22,

Recreating large image for HTA11_411_20000010115220260010000000000.tif




56it [00:30,  1.49it/s]



Recreating large image for HTA11_411_20000010115220260020000000000.tif


57it [00:31,  1.26it/s]

58it [00:32,  1.10it/s]

Recreating large image for HTA11_411_20000010115220260030000000000.tif
Recreating large image for HTA11_411_20000010115220260040000000000.tif




59it [00:34,  1.01s/it]

60it [00:41,  2.81s/it]

61it [00:41,  2.10s/it]

62it [00:47,  3.11s/it]

63it [00:47,  2.30s/it]

64it [00:47,  1.75s/it]

65it [00:48,  1.36s/it]

66it [00:48,  1.09s/it]

67it [00:49,  1.12it/s]

68it [00:49,  1.23it/s]

69it [00:50,  1.32it/s]

70it [00:51,  1.49it/s]

71it [00:51,  1.58it/s]

72it [00:51,  1.76it/s]

73it [00:52,  1.86it/s]

74it [00:52,  1.93it/s]

75it [00:53,  1.97it/s]

76it [00:53,  1.98it/s]

77it [00:54,  2.05it/s]

78it [00:54,  2.12it/s]

79it [00:55,  2.19it/s]

80it [00:55,  2.19it/s]

81it [00:56,  2.23it/s]

82it [00:56,  2.04it/s]

83it [00:57,  2.11it/s]

84it [00:57,  2.14it/s]

85it [00:57,  2.24it/s]

86it [00:58,  1.84it/s]

87it [00:59,  1.97it/s]

88it [00:59,  2.02it/s]

89it [01:00,  2.05it/s]

90it [01:00,  2.05it/s]

91it [01:01,  2.09it/s]

92it [01:02,  1.51it/s]

Recreating large image for HTA11_866_20000010115220260010000000000.tif
Recreating large image for HTA11_866_20000010115220260020000000000.tif




93it [01:03,  1.16it/s]

In [21]:
def checkForLargeImage( i):
    ## Given a girder item, see if it has a largeImage tag
    ## If it does, also check it the image has frames... if not try and force it to be recreated
    if 'largeImage' in i:
        ### Check and see if the item has frames
        ## Scan for gdal here as well
        
        tileInfo = gc.get("/item/%s/tiles" % i['_id']) 
        if 'frames' not in tileInfo:
            ## found a single frame image... should check and see if the tile source is gdal
            ### Maybe add ioparams info that image was reconverted?
            gc.delete("item/%s/tiles" % i['_id'])
            gc.addMetadataToItem(i['_id'],{'ioparams':{"forceLargeImage":True}})
            gc.post("/item/%s/tiles?force=true" % i['_id'])
            print("Recreating large image for", i['name'])
    else:
        gc.addMetadataToItem(i['_id'],{'ioparams':{"forceLargeImage":True}})
        gc.post("/item/%s/tiles?force=true" % i['_id'])
        print("Recreating large image for", i['name'])
        

In [3]:
def analyzeIoParams( item, csvData=None, channelMap = None, updateItem=True,zeroIndexed=False):
    ### IoParams stores the necessary met        #print("Found metadata for",dataFileId)
    ##adata to view a multiplex image in the DSA for the multiplex viewer
    ## Ioparams has a number of different functionality to enhance the visualization, most importantly being the 
    ## channel names
    ## I am going to compare the new ioparams to the one that currently exists for the item
    ## Depending on what additional params are passed, I will integrate all the processing in here
    ## in terms of processing the csvData or channel maps, or adding the default color palette, min /max /etc
    
    
    ## IF the input metadata file is NOT zeroindexed, i.e. the frame starts at one and not 0.. I need to subtract
    ## one from each frame so the channels are mapped proerly
    
    i = gc.getItem( item['_id']) ## Refresh the item and get the latest metadata..
    
    ioparams = i['meta'].get('ioparams',{})

    ## Parsing the csv metadata file and grabbing the channel map.
    if csvData:
        channelMap = {}
        for r in csvData:
            channelMap[r['MARKERNAME']] = int(r['LAYER']) -1 ## frames start at 0 or 1...
        ioparams['channelmap'] = channelMap 
        
    if updateItem:
        gc.addMetadataToItem(item['_id'], {'ioparams': ioparams})
    
    return ioparams 

In [20]:
def analyzeVandyMetadata( i):
    ## For now, I will add metadata from frame0 of the CSV metadata file, for now will maybe ad it to
    #  vandySlideMeta

    frameZeroMetaKeys =['Component', 'Filename', 'File Format', 'HTAN Participant ID', 'HTAN Parent Biospecimen ID', 'HTAN Data File ID', 'Channel Metadata Filename', 'Imaging Assay Type', 'Protocol Link', 'Workflow Start Datetime', 'Workflow End Datetime', 'Software and Version', 'Microscope', 'Objective', 'NominalMagnification', 'LensNA', 'WorkingDistance', 'WorkingDistanceUnit', 'Immersion', 'Pyramid', 'Zstack', 'Tseries', 'Passed QC', 'Comment', 'FOV number', 'FOVX', 'FOVXUnit', 'FOVY', 'FOVYUnit', 'Frame Averaging', 'Image ID', 'DimensionOrder', 'PhysicalSizeX', 'PhysicalSizeXUnit', 'PhysicalSizeY', 'PhysicalSizeYUnit', 'PhysicalSizeZ', 'PhysicalSizeZUnit', 'Pixels BigEndian', 'PlaneCount', 'SizeC', 'SizeT', 'SizeX', 'SizeY', 'SizeZ', 'PixelType', 'LEVEL', 'TYPE', 'LAYERS', 'REGION', 'POSITION', 'LAYER', 'ROUND', 'CHANNEL', 'MARKERID', 'MARKERNAME', 'AbID', 'CLONE', 'DYE', 'VENDOR', 'CATNUM', 'EXPOSURE(ms)', 'DILUTION']




In [5]:
for i in tqdm.tqdm( gc.listItem(vandyMxIfFolder)):
    gc.addMetadataToItem(i['_id'], {'dataSet': '2022reload'})


93it [00:11, 10.52it/s]


In [18]:
vmdCache[dataFileId]

[{'Component': 'ImagingLevel2',
  'Filename': 'mxif_level_2/AFsubtracted/HTA11_1391_20000010115220280010000000000.tif',
  'File Format': 'tif',
  'HTAN Participant ID': 'HTA11_1391',
  'HTAN Parent Biospecimen ID': 'HTA11_1391_2000001011',
  'HTAN Data File ID': 'HTA11_1391_20000010115220280010000000000',
  'Channel Metadata Filename': 'mxif_level_2/AFsubtracted/metadata/HTA11_1391_20000010115220280010000000000_metadata.csv',
  'Imaging Assay Type': 'MxIF',
  'Protocol Link': 'NONE',
  'Workflow Start Datetime': '10/01/2020',
  'Workflow End Datetime': '10/02/2020',
  'Software and Version': 'ImageApp 1.0.0.0',
  'Microscope': 'INCELL ANALYZER 2500 HS',
  'Objective': 'NIKON MRD00205 Plan Apochromat',
  'NominalMagnification': '20X',
  'LensNA': '0.75',
  'WorkingDistance': '1',
  'WorkingDistanceUnit': 'mm',
  'Immersion': 'Air',
  'Pyramid': 'No',
  'Zstack': 'Yes',
  'Tseries': 'No',
  'Passed QC': 'Yes',
  'Comment': 'NaN',
  'FOV number': 'NaN',
  'FOVX': 'NaN',
  'FOVXUnit': 'NaN